In [3]:
import numpy as np
import tensorflow as tf
import keras
import os

In [5]:
curr_path = os.getcwd() + '/'
PROCESSED_DATA_PATH = os.path.join(curr_path,'origin_npy')
save_signals_path_SC = os.path.join(PROCESSED_DATA_PATH,'signals_SC_filtered')
save_annotations_path_SC = os.path.join(PROCESSED_DATA_PATH,'annotations_SC')
save_signals_path_ST = os.path.join(PROCESSED_DATA_PATH,'signals_ST_filtered')
save_annotations_path_ST = os.path.join(PROCESSED_DATA_PATH,'annotations_ST')

In [6]:
def match_annotations_npy(dirname, filename):
    search_filename = filename.split('-')[0][:-2]
    file_list = os.listdir(dirname)
    filenames = [file for file in file_list if search_filename in file if file.endswith('.npy')]

    return filenames

In [7]:
dim_HT1D = (3000,1)
n_classes=6
epochs = 50
bs = 64
list_files = [f for f in os.listdir(save_signals_path_ST) if f.endswith('.npy')]
list_ann_files = []
for f in list_files:
    ann_file = match_annotations_npy(save_annotations_path_ST, f)
    list_ann_files.append(ann_file[0])

In [8]:
split_cnt = int(len(list_files) * 0.8)
list_files_train = list_files[:split_cnt]
list_files_test = list_files[split_cnt:]
list_ann_files_train = list_ann_files[:split_cnt]
list_ann_files_test = list_ann_files[split_cnt:]

#list_files_train = list_files[:5]
#list_files_test = list_files[80:90]
#list_ann_files_train = list_ann_files[0:5]
#list_ann_files_test = list_ann_files[80:90]

In [9]:
train_generator = DataGenerator(save_signals_path_ST, save_annotations_path_ST, list_files_train, list_ann_files_train, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False)

In [10]:
test_generator = DataGenerator(save_signals_path_ST, save_annotations_path_ST, list_files_test, list_ann_files_test, 
                          batch_size=bs, dim=dim_HT1D, n_classes=n_classes, shuffle=False)

In [11]:
list_ann_files_train

['ST7202JO-Hypnogram.npy',
 'ST7082JW-Hypnogram.npy',
 'ST7062JR-Hypnogram.npy',
 'ST7152JA-Hypnogram.npy',
 'ST7072JA-Hypnogram.npy',
 'ST7121JE-Hypnogram.npy',
 'ST7172JA-Hypnogram.npy',
 'ST7041JO-Hypnogram.npy',
 'ST7242JO-Hypnogram.npy',
 'ST7201JO-Hypnogram.npy',
 'ST7222JA-Hypnogram.npy',
 'ST7171JA-Hypnogram.npy',
 'ST7112JE-Hypnogram.npy',
 'ST7212JJ-Hypnogram.npy',
 'ST7051JA-Hypnogram.npy',
 'ST7012JP-Hypnogram.npy',
 'ST7101JE-Hypnogram.npy',
 'ST7192JR-Hypnogram.npy',
 'ST7241JO-Hypnogram.npy',
 'ST7061JR-Hypnogram.npy',
 'ST7161JM-Hypnogram.npy',
 'ST7182JR-Hypnogram.npy',
 'ST7211JJ-Hypnogram.npy',
 'ST7162JM-Hypnogram.npy',
 'ST7052JA-Hypnogram.npy',
 'ST7181JR-Hypnogram.npy',
 'ST7081JW-Hypnogram.npy',
 'ST7091JE-Hypnogram.npy',
 'ST7011JP-Hypnogram.npy']

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=dim_HT1D),
    keras.layers.Conv1D(filters=64, kernel_size=100, padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    keras.layers.Conv1D(filters=128, kernel_size=100, padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    keras.layers.Conv1D(filters=128, kernel_size=100, padding="valid"),
    tf.keras.layers.BatchNormalization(axis=-1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(n_classes, activation="softmax")
])

2022-02-14 23:14:53.671405: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-02-14 23:14:54.466961: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:25:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-02-14 23:14:54.467930: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-02-14 23:14:55.407321: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-02-14 23:14:55.407388: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-02-14 23:14:55.565447: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcu

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 2901, 64)          6464      
_________________________________________________________________
batch_normalization (BatchNo (None, 2901, 64)          256       
_________________________________________________________________
re_lu (ReLU)                 (None, 2901, 64)          0         
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 2802, 128)         819328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 2802, 128)         512       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 2802, 128)         0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 2703, 128)         1

In [14]:
callbacks = []

#Checkpoint설정
checkpoint_dir = './ckpt_conv1D_Raw'
model_cp_path = os.path.join(checkpoint_dir, "ckpt_{epoch}")
callbacks.append(tf.keras.callbacks.ModelCheckpoint(model_cp_path, save_weights_only=True))


#Learning rate 스케쥴 설정
#callbacks.append(LearningRateScheduler(decay))

#General logs on csv
#callbacks.append(CSVLogger(model_csv_path)) 

In [15]:
def eval_model(test_gen, model):
    total_cnt = 0.0
    total_loss = 0.0
    correct = 0.0
    for x, y in test_gen:
        y_pred = model.predict(x)
        y_pred_cls = tf.math.argmax(y_pred, axis=-1)
        correct += tf.reduce_sum(tf.cast(tf.equal(y_pred_cls, y), tf.float32))
        total_cnt += y_pred.shape[0]
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, 
                                                                reduction=tf.keras.losses.Reduction.SUM)
        y = tf.cast(y, dtype=tf.int32)
        total_loss += loss_fn(y, y_pred).numpy()
            
    test_acc = correct / total_cnt
    test_loss = total_loss / total_cnt
    
    print("test_acc: %.3f, test_loss: %.6f"%(test_acc, test_loss))

In [16]:
class eval_callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        eval_model(test_generator, model)

In [17]:
callbacks.append(eval_callback())

In [18]:
model.compile(
    optimizer="sgd",
    loss="sparse_categorical_crossentropy",
    metrics=['sparse_categorical_accuracy'],
)

In [ ]:
history = model.fit(train_generator,              
              epochs=epochs,
              verbose=1,
              callbacks=callbacks)

2022-02-14 23:15:01.386967: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-14 23:15:01.421309: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2499920000 Hz


Epoch 1/50


2022-02-14 23:15:02.386652: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-02-14 23:15:04.863636: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2022-02-14 23:15:09.027480: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-14 23:15:09.027548: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2022-02-14 23:15:09.061043: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-14 23:15:09.061261: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-02-14 23:15:09.149107: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

447/447 [==============================] - 37s 54ms/step - loss: 5.8979 - sparse_categorical_accuracy: 0.4331
test_acc: 0.412, test_loss: 1.619471
Epoch 2/50
447/447 [==============================] - 24s 54ms/step - loss: 1.3144 - sparse_categorical_accuracy: 0.5007
test_acc: 0.435, test_loss: 1.443910
Epoch 3/50
447/447 [==============================] - 24s 54ms/step - loss: 1.2617 - sparse_categorical_accuracy: 0.5145
test_acc: 0.463, test_loss: 1.382097
Epoch 4/50
447/447 [==============================] - 24s 54ms/step - loss: 1.2434 - sparse_categorical_accuracy: 0.5100
test_acc: 0.444, test_loss: 1.406881
Epoch 5/50
447/447 [==============================] - 24s 54ms/step - loss: 1.2281 - sparse_categorical_accuracy: 0.5170
test_acc: 0.303, test_loss: 1.996570
Epoch 6/50
447/447 [==============================] - 24s 54ms/step - loss: 1.2086 - sparse_categorical_accuracy: 0.5235
test_acc: 0.435, test_loss: 1.474158
Epoch 7/50
447/447 [==============================] - 24s 54ms/

In [ ]:
x = y

In [17]:
# RUn result with no filters applied
# Best test acc: 0.525
"""
history = model.fit(train_generator,              
              epochs=epochs,
              verbose=1,
              callbacks=callbacks)
"""

2022-02-14 19:01:38.631039: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-14 19:01:38.677490: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2500295000 Hz


Epoch 1/50


2022-02-14 19:01:39.746985: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-02-14 19:01:40.321041: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2022-02-14 19:01:41.251069: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-14 19:01:41.251142: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2022-02-14 19:01:41.252954: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-14 19:01:41.253048: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-02-14 19:01:41.286455: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

447/447 [==============================] - 29s 53ms/step - loss: 4.1422 - sparse_categorical_accuracy: 0.4332
test_acc: 0.425, test_loss: 1.770779
Epoch 2/50
447/447 [==============================] - 24s 54ms/step - loss: 1.4555 - sparse_categorical_accuracy: 0.4982
test_acc: 0.360, test_loss: 1.954212
Epoch 3/50
447/447 [==============================] - 24s 54ms/step - loss: 1.3464 - sparse_categorical_accuracy: 0.4945
test_acc: 0.443, test_loss: 1.746703
Epoch 4/50
447/447 [==============================] - 24s 53ms/step - loss: 1.2835 - sparse_categorical_accuracy: 0.4992
test_acc: 0.429, test_loss: 1.812051
Epoch 5/50
447/447 [==============================] - 24s 53ms/step - loss: 1.2335 - sparse_categorical_accuracy: 0.5206
test_acc: 0.433, test_loss: 1.688398
Epoch 6/50
447/447 [==============================] - 24s 54ms/step - loss: 1.2277 - sparse_categorical_accuracy: 0.5122
test_acc: 0.478, test_loss: 1.598330
Epoch 7/50
447/447 [==============================] - 24s 54ms/